In [ ]:
import pandas as pd
import numpy as np
import os
import random
from tqdm import tqdm
from pathlib import Path
import cv2 as cv
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import layers, optimizers, losses, metrics, callbacks
from tensorflow.keras import Sequential, Model, Input
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings("ignore")
random.seed(45)
tf.random.set_seed(45)
np.random.seed(45)

print(tf.__version__)
import glob
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
# Load metadata to get age information
HOME_PATH = os.getcwd() + "/"
meta_train = pd.read_csv(HOME_PATH + 'meta_train.csv')

# Create mapping from image_id to age
image_to_age = dict(zip(meta_train['image_id'], meta_train['age']))

# Load images with age labels
def load_images_with_age(paths):
    data = []
    labels = []
    i = 0
    for label, path in tqdm(enumerate(paths)):
        for img_path in os.listdir(path):
            image = np.array(Image.open(os.path.join(path,img_path)).convert('RGB').resize((256,256)))
            data.append(image)
            labels.append(label)
    return np.array(data), np.asarray(labels)

# Load all training images
print("Loading training images...")
images, ages = load_images_with_age(glob.glob(HOME_PATH + 'train_images/*'))

print(f"Loaded {len(images)} images")
print(f"Age range: {ages.min()} to {ages.max()} days")
print(f"Average age: {ages.mean():.2f} days")

In [ ]:
# Configuration
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 64
num_epochs = 100
image_size = 72
patch_size = 6
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
]
transformer_layers = 8
mlp_head_units = [2048, 1024]
k_folds = 5  # Number of folds

In [ ]:
# MLP helper function
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

# Patches layer
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

# Patch encoder
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded



2.16.1
Loading training images...


10it [01:05,  6.58s/it]


Loaded 10407 images
Age range: 0 to 9 days
Average age: 5.39 days


In [ ]:
# ViT Model for Age Regression with adjustable input shape for preprocessing
def create_vit_regressor(input_images):
    # Create data augmentation inside model
    data_augmentation = keras.Sequential(
        [
            layers.Normalization(),
            layers.Resizing(image_size, image_size),
            layers.RandomFlip("horizontal"),
            layers.RandomRotation(factor=0.02),
            layers.RandomZoom(height_factor=0.2, width_factor=0.2),
        ],
        name="data_augmentation",
    )
    
    # Adapt normalization layer to current fold's data
    data_augmentation.layers[0].adapt(input_images)
    
    inputs = layers.Input(shape=(256, 256, 3))
    
    # Augment data
    augmented = data_augmentation(inputs)
    
    # Create patches
    patches = Patches(patch_size)(augmented)
    
    # Encode patches
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Create multiple layers of the Transformer block
    for _ in range(transformer_layers):
        # Layer normalization 1
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        
        # Create a multi-head attention layer
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        
        # Skip connection 1
        x2 = layers.Add()([attention_output, encoded_patches])
        
        # Layer normalization 2
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        
        # MLP
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        
        # Skip connection 2
        encoded_patches = layers.Add()([x3, x2])

    # Create a [batch_size, projection_dim] tensor
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)
    
    # Add MLP
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
    
    # Output layer for regression (single neuron for age)
    output = layers.Dense(1, activation='linear')(features)
    
    # Create the Keras model
    model = keras.Model(inputs=inputs, outputs=output)
    return model

In [ ]:
# K-Fold Cross Validation
def perform_kfold_training():
    kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)
    
    fold_results = []
    fold_histories = []
    ensemble_predictions = []
    
    # Create directory for k-fold models
    kfold_model_dir = HOME_PATH + 'paddy_models/kfold_models/'
    os.makedirs(kfold_model_dir, exist_ok=True)
    
    print(f"\nStarting {k_folds}-Fold Cross Validation...")
    
    for fold, (train_idx, val_idx) in enumerate(kfold.split(images), 1):
        print(f"\n{'='*50}")
        print(f"Training Fold {fold}/{k_folds}")
        print(f"{'='*50}")
        
        # Split data for current fold
        X_train_fold, X_val_fold = images[train_idx], images[val_idx]
        y_train_fold, y_val_fold = ages[train_idx], ages[val_idx]
        
        # Calculate age statistics for current fold
        age_mean_fold = y_train_fold.mean()
        age_std_fold = y_train_fold.std()
        
        # Normalize ages for current fold
        y_train_norm_fold = (y_train_fold - age_mean_fold) / age_std_fold
        y_val_norm_fold = (y_val_fold - age_mean_fold) / age_std_fold
        
        print(f"Training samples: {len(X_train_fold)}")
        print(f"Validation samples: {len(X_val_fold)}")
        print(f"Age mean: {age_mean_fold:.2f}, std: {age_std_fold:.2f}")
        
        # Create model for current fold
        model = create_vit_regressor(X_train_fold)
        
        # Compile model
        optimizer = keras.optimizers.AdamW(
            learning_rate=learning_rate, weight_decay=weight_decay
        )
        
        model.compile(
            optimizer=optimizer,
            loss='mean_absolute_error',
            metrics=['mae', 'mse']
        )
        
        # Callbacks
        filepath = f"{kfold_model_dir}best_vit_age_model_fold_{fold}.keras"
        
        checkpoint = tf.keras.callbacks.ModelCheckpoint(
            filepath=filepath,
            monitor="val_loss",
            verbose=1,
            save_best_only=True,
            mode='min'
        )
        
        reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss",
            factor=0.2,
            patience=8,
            verbose=1,
            mode="min",
            min_delta=0.001,
            cooldown=2,
            min_lr=1e-7
        )
        
        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=20,
            verbose=1,
            mode="min",
            restore_best_weights=True
        )
        
        # Train model for current fold
        history = model.fit(
            x=X_train_fold,
            y=y_train_norm_fold,
            batch_size=batch_size,
            epochs=num_epochs,
            validation_data=(X_val_fold, y_val_norm_fold),
            callbacks=[checkpoint, reduce_lr, early_stopping],
            verbose=1
        )
        
        # Evaluate on validation set
        val_predictions = model.predict(X_val_fold)
        val_predictions_original = val_predictions.flatten() * age_std_fold + age_mean_fold
        y_val_original = y_val_fold
        
        mae = mean_absolute_error(y_val_original, val_predictions_original)
        mse = mean_squared_error(y_val_original, val_predictions_original)
        rmse = np.sqrt(mse)
        
        # Store results
        fold_result = {
            'fold': fold,
            'mae': mae,
            'rmse': rmse,
            'mse': mse,
            'age_mean': age_mean_fold,
            'age_std': age_std_fold,
            'best_val_loss': min(history.history['val_loss']),
            'final_val_loss': history.history['val_loss'][-1]
        }
        fold_results.append(fold_result)
        fold_histories.append(history)
        
        # Load the best model for this fold and make predictions on all test data
        best_model = keras.models.load_model(filepath)
        ensemble_predictions.append((best_model, age_mean_fold, age_std_fold))
        
        print(f"\nFold {fold} Results:")
        print(f"Validation MAE: {mae:.2f} days")
        print(f"Validation RMSE: {rmse:.2f} days")
        print(f"Best validation loss: {fold_result['best_val_loss']:.4f}")
        
        # Clean up model from memory
        del model
        del best_model
        tf.keras.backend.clear_session()
    
    return fold_results, fold_histories, ensemble_predictions

In [ ]:
# Perform k-fold training
fold_results, fold_histories, ensemble_predictions = perform_kfold_training()

# Analyze k-fold results
print("\n" + "="*50)
print("K-Fold Cross Validation Results")
print("="*50)

mae_scores = [result['mae'] for result in fold_results]
rmse_scores = [result['rmse'] for result in fold_results]

print(f"\nMAE per fold: {mae_scores}")
print(f"RMSE per fold: {rmse_scores}")
print(f"\nAverage MAE: {np.mean(mae_scores):.2f} ± {np.std(mae_scores):.2f}")
print(f"Average RMSE: {np.mean(rmse_scores):.2f} ± {np.std(rmse_scores):.2f}")


Starting 5-Fold Cross Validation...

Training Fold 1/5
Training samples: 8325
Validation samples: 2082
Age mean: 5.40, std: 2.55

Epoch 1/100
131/131 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 2.4424 - mae: 2.4424 - mse: 20.9888
Epoch 1: val_loss improved from inf to 0.81215, saving model to c:\Users\ThinkPad\Desktop\COSC2753_A2_MachineLearning/paddy_models/kfold_models/best_vit_age_model_fold_1.keras
131/131 ━━━━━━━━━━━━━━━━━━━━ 193s 1s/step - loss: 2.4341 - mae: 2.4341 - mse: 20.8773 - val_loss: 0.8121 - val_mae: 0.8121 - val_mse: 0.9410 - learning_rate: 0.0010
Epoch 2/100
131/131 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 0.8424 - mae: 0.8424 - mse: 1.0976
Epoch 2: val_loss improved from 0.81215 to 0.74701, saving model to c:\Users\ThinkPad\Desktop\COSC2753_A2_MachineLearning/paddy_models/kfold_models/best_vit_age_model_fold_1.keras
131/131 ━━━━━━━━━━━━━━━━━━━━ 177s 1s/step - loss: 0.8423 - mae: 0.8423 - mse: 1.0973 - val_loss: 0.7470 - val_mae: 0.7470 - val_mse: 0.8980 - learning_rate: 

In [ ]:
# Save k-fold results
results_df = pd.DataFrame(fold_results)
results_df.to_csv(HOME_PATH + 'kfold_results.csv', index=False)

# Plot k-fold training history
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('K-Fold Training History', fontsize=16)

# Plot loss
axes[0, 0].set_title('Training Loss by Fold')
for i, history in enumerate(fold_histories):
    axes[0, 0].plot(history.history['loss'], label=f'Fold {i+1}', alpha=0.7)
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss (MAE)')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Plot validation loss
axes[0, 1].set_title('Validation Loss by Fold')
for i, history in enumerate(fold_histories):
    axes[0, 1].plot(history.history['val_loss'], label=f'Fold {i+1}', alpha=0.7)
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Validation Loss (MAE)')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Plot MAE
axes[1, 0].set_title('Training MAE by Fold')
for i, history in enumerate(fold_histories):
    axes[1, 0].plot(history.history['mae'], label=f'Fold {i+1}', alpha=0.7)
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('MAE')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Plot validation MAE
axes[1, 1].set_title('Validation MAE by Fold')
for i, history in enumerate(fold_histories):
    axes[1, 1].plot(history.history['val_mae'], label=f'Fold {i+1}', alpha=0.7)
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Validation MAE')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(HOME_PATH + 'kfold_training_history.png', dpi=300, bbox_inches='tight')
plt.show()

# Plot fold comparison
plt.figure(figsize=(10, 6))
folds = [f'Fold {i+1}' for i in range(k_folds)]
x = np.arange(len(folds))
width = 0.35

plt.bar(x - width/2, mae_scores, width, label='MAE', alpha=0.8)
plt.bar(x + width/2, rmse_scores, width, label='RMSE', alpha=0.8)

plt.ylabel('Error (days)')
plt.title('Performance Comparison Across Folds')
plt.xticks(x, folds)
plt.legend()
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.savefig(HOME_PATH + 'kfold_performance_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

# Generate ensemble predictions for test set
def load_test_images():
    test_data = []
    test_ids = []
    
    test_path = HOME_PATH + 'test_images/'
    for img_file in tqdm(os.listdir(test_path)):
        if img_file.endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(test_path, img_file)
            image = np.array(Image.open(img_path).convert('RGB').resize((256, 256)))
            test_data.append(image)
            test_ids.append(img_file.split('.')[0])
    
    return np.array(test_data), test_ids

# Load test images
print("\nLoading test images...")
test_images, test_ids = load_test_images()

# Ensemble prediction
print("\nMaking ensemble predictions...")
all_predictions = []

for model, age_mean, age_std in ensemble_predictions:
    predictions_norm = model.predict(test_images, verbose=0)
    predictions_original = predictions_norm.flatten() * age_std + age_mean
    all_predictions.append(predictions_original)

# Average predictions across folds
ensemble_predictions_avg = np.mean(all_predictions, axis=0)

# Create submission dataframe
submission_df = pd.DataFrame({
    'image_id': test_ids,
    'age': ensemble_predictions_avg.astype(int).astype(str)
})

# Save ensemble predictions
submission_df.to_csv(HOME_PATH + 'age_predictions_kfold_ensemble.csv', index=False)

print(f"\nEnsemble prediction statistics:")
print(f"Min predicted age: {ensemble_predictions_avg.min():.2f} days")
print(f"Max predicted age: {ensemble_predictions_avg.max():.2f} days")
print(f"Mean predicted age: {ensemble_predictions_avg.mean():.2f} days")

In [ ]:
# Also save individual fold predictions for analysis
individual_predictions_df = pd.DataFrame({
    'image_id': test_ids,
    **{f'fold_{i+1}_prediction': pred for i, pred in enumerate(all_predictions)}
})
individual_predictions_df.to_csv(HOME_PATH + 'age_predictions_individual_folds.csv', index=False)

# Analyze prediction variance across folds
prediction_std = np.std(all_predictions, axis=0)
prediction_variance_df = pd.DataFrame({
    'image_id': test_ids,
    'ensemble_prediction': ensemble_predictions_avg,
    'prediction_std': prediction_std,
    'coefficient_of_variation': prediction_std / ensemble_predictions_avg
})
prediction_variance_df.to_csv(HOME_PATH + 'prediction_variance_analysis.csv', index=False)

print(f"\nPrediction variance statistics:")
print(f"Average standard deviation across predictions: {prediction_std.mean():.2f}")
print(f"Maximum standard deviation: {prediction_std.max():.2f}")
print(f"Images with high prediction variance (std > 10): {sum(prediction_std > 10)}")

# Save age statistics (using average from all folds)
age_stats = {
    'mean': np.mean([result['age_mean'] for result in fold_results]),
    'std': np.mean([result['age_std'] for result in fold_results])
}

import json
with open(HOME_PATH + 'age_stats_kfold.json', 'w') as f:
    json.dump(age_stats, f)

print("\nK-fold cross-validation completed successfully!")